In [1]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "you hugging face api token"

In [2]:
!pip install langchain chromadb langchain_huggingface tiktoken pypdf langchain-community ChatHuggingFace HuggingFaceEndpoint

ERROR: Could not find a version that satisfies the requirement ChatHuggingFace (from versions: none)
ERROR: No matching distribution found for ChatHuggingFace


In [3]:
!pip install langchain_huggingface

  Using cached langchain_huggingface-0.3.1-py3-none-any.whl.metadata (996 bytes)


In [4]:
!pip install langchain_community

  Using cached langchain_community-0.3.29-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [18]:
!pip install -qU langchain-community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 62.4 MB/s eta 0:00:00


In [23]:
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain.vectorstores import FAISS


In [24]:
from langchain.schema import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [25]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [26]:
import faiss

In [37]:
vector_store = FAISS.from_documents(
    docs,
    HuggingFaceEndpointEmbeddings(model="sentence-transformers/all-MiniLM-L6-v2")
)


In [52]:
# add documents
vector_store.add_documents(docs)

['9054516b-de89-47b2-befa-10a53e44a3d3',
 '9b2b22bb-0cf2-4bb7-a989-62fe53cbd4f9',
 '3010d25c-343d-4c73-ac2f-445f3e1436fe',
 '233d921e-47b1-4fc0-94ec-8c446afd0caf',
 'c69dadad-40bc-44f5-aa45-79cd73542603']

In [53]:
# Access all documents stored in FAISS
all_docs = vector_store.docstore._dict

In [54]:
for doc_id, doc in all_docs.items():
    print(f"ID: {doc_id}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

ID: dcbbf777-9d4e-479b-bb1f-d512e23bb02e
Content: Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
ID: c6222e65-3970-4d7a-b89b-ed83954e342e
Content: MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.
Metadata: {'team': 'Chennai Super Kings'}
--------------------------------------------------
ID: 01b09bd1-0139-48b1-b95e-f10559931c8b
Content: Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
ID: 5d51aca1-2032-4e93-9a14-43ec7da11a8f
Content: Ravindra Jadeja is a dynamic all-r

In [55]:
# search documents
vector_store.similarity_search(
    query='Who among these are a bowler?',
    k=2
)

[Document(id='01b09bd1-0139-48b1-b95e-f10559931c8b', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
 Document(id='4730c5a4-fe54-42ee-a1b6-7939856810c9', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.')]

In [56]:
# search with similarity score
vector_store.similarity_search_with_score(
    query='Who among these are a bowler?',
    k=2
)

[(Document(id='01b09bd1-0139-48b1-b95e-f10559931c8b', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  np.float32(0.9693603)),
 (Document(id='4730c5a4-fe54-42ee-a1b6-7939856810c9', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  np.float32(0.9693603))]

In [57]:
# meta-data filtering
vector_store.similarity_search_with_score(
    query="",
    filter={"team": "Chennai Super Kings"}
)

[(Document(id='c6222e65-3970-4d7a-b89b-ed83954e342e', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  np.float32(1.8436004)),
 (Document(id='bb04328e-336a-457b-80ea-c120304c8372', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  np.float32(1.8436004)),
 (Document(id='3010d25c-343d-4c73-ac2f-445f3e1436fe', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  np.float32(1.8436004)),
 (Document(id='5d51aca1-2032-4e93-9a14-43ec7da11a8f', metadata={'team': 'Chennai Super Kings'}, page

In [58]:
# update documents
# updated doc
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

# Step 1: Delete the old doc by ID (if you know it)
vector_store.delete(['00426e74-22fc-425d-87a3-be76aa58b024'])

# Step 2: Add the updated document
vector_store.add_documents([updated_doc1])

['1956bec4-58b0-4465-9016-5be442849c02']

In [62]:
# Access all documents stored in FAISS
all_docs = vector_store.docstore._dict
for doc_id, doc in all_docs.items():
    print(f"ID: {doc_id}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

ID: dcbbf777-9d4e-479b-bb1f-d512e23bb02e
Content: Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
ID: c6222e65-3970-4d7a-b89b-ed83954e342e
Content: MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.
Metadata: {'team': 'Chennai Super Kings'}
--------------------------------------------------
ID: 01b09bd1-0139-48b1-b95e-f10559931c8b
Content: Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
ID: 5d51aca1-2032-4e93-9a14-43ec7da11a8f
Content: Ravindra Jadeja is a dynamic all-r

In [64]:
# delete document
vector_store.delete(ids=['1956bec4-58b0-4465-9016-5be442849c02'])

True

In [65]:
# Access all documents stored in FAISS
all_docs = vector_store.docstore._dict
for doc_id, doc in all_docs.items():
    print(f"ID: {doc_id}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 50)

ID: dcbbf777-9d4e-479b-bb1f-d512e23bb02e
Content: Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
ID: c6222e65-3970-4d7a-b89b-ed83954e342e
Content: MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.
Metadata: {'team': 'Chennai Super Kings'}
--------------------------------------------------
ID: 01b09bd1-0139-48b1-b95e-f10559931c8b
Content: Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.
Metadata: {'team': 'Mumbai Indians'}
--------------------------------------------------
ID: 5d51aca1-2032-4e93-9a14-43ec7da11a8f
Content: Ravindra Jadeja is a dynamic all-r